# Valuing Caps and Floors

In [1]:
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the FinancePy license for more details.

We value caps and floors using Black's model

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from financepy.finutils import *
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.products.libor.FinLiborCapFloor import *
from financepy.market.curves import *

## Building a Libor Curve

In [7]:
valuationDate = FinDate(6, 6, 2018)

In [8]:
spotDays = 0
settlementDate = valuationDate.addWorkDays(spotDays)
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = FinLiborDeposit(settlementDate, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "9M", 0.0230, depoDCCType); depos.append(depo)

In [9]:
accrual = FinDayCountTypes.THIRTY_360
freq = FinFrequencyTypes.SEMI_ANNUAL
longEnd = FinDateGenRuleTypes.BACKWARD

spotDays = 2
settlementDate = valuationDate.addWorkDays(spotDays)

swaps = []
swap = FinLiborSwap(settlementDate, "1Y", 0.0250, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "2Y", 0.0255, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "3Y", 0.0260, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", 0.0265, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", 0.0270, freq, accrual); swaps.append(swap)

In [10]:
liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

## Creating a Cap and Floor

In [11]:
capType = FinLiborCapFloorType.CAP
floorType = FinLiborCapFloorType.FLOOR

In [12]:
strikeRate = 0.02

In [13]:
cap = FinLiborCapFloor(settlementDate, "2Y", capType, strikeRate)
flr = FinLiborCapFloor(settlementDate, "2Y", floorType, strikeRate)

In [14]:
print(cap)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorType.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


In [15]:
print(flr)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorType.FLOOR
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


## Valuation

### Black's Model

We start with Black's model with 25% volatility

In [16]:
model = FinModelBlack(0.25)

In [17]:
cap.value(valuationDate, liborCurve, model)

11373.487460663488

In [18]:
flr.value(valuationDate, liborCurve, model)

839.0297040488564

In [20]:
cap.printLeg()

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
OPTION TYPE FinLiborCapFloorType.CAP
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE WED 6 JUN 2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
 FRI 8 JUN 2018  0.0000000    0.00000         0.00     1.000000         0.00         0.00
MON 10 SEP 2018  0.2555556    2.34863       885.63     0.994034       885.63       885.63
MON 10 DEC 2018  0.2500000    2.28739       710.13     0.988382       761.48      1647.11
 FRI 8 MAR 2019  0.2444444    2.28951       695.58     0.982881       814.60      2461.71
MON 10 JUN 2019  0.2555556    3.03491      2579.49     0.975316      2592.95      5054.66
 MON 9 SEP 2019  0.2472222    2.60810      1456.85     0.969068      1558.86      6613.52
 MON 9 DEC 2019  0.2500000    2.57912      1394.03     0.962860      1545.14      8158.66
 MON 9 MAR 2020  0.2500000    2.57912      1385.10     

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [21]:
model = FinModelBlackShifted(0.25, -0.005)

In [22]:
cap.value(valuationDate, liborCurve, model)

11907.567907413986

In [23]:
flr.value(valuationDate, liborCurve, model)

1373.111675229057

The floor has increased in value as the downside risk is greater.

## SABR Model

In [24]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [25]:
model = FinModelSABR(alpha, beta, rho, nu)

In [26]:
cap.value(valuationDate, liborCurve, model)

11476.08433550521

In [27]:
flr.value(valuationDate, liborCurve, model)

941.6247561755903

## Shifted SABR Model

In [28]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [29]:
model = FinModelSABRShifted(alpha, beta, rho, nu, shift)

In [30]:
cap.value(valuationDate, liborCurve, model)

11844.971927506947

In [31]:
flr.value(valuationDate, liborCurve, model)

1310.512348177328

## Hull White Model

In [32]:
sigma = 0.01; alpha = 0.005

In [33]:
model = FinModelRatesHW(sigma, alpha)

In [34]:
cap.value(valuationDate, liborCurve, model)

11149.637416506732

In [35]:
flr.value(valuationDate, liborCurve, model)

593.6128741545962

Copyright (c) 2020 Dominic O'Kane